In [9]:
import cv2
import numpy as np
import itertools

# Comparator

In [19]:
def image_comparator(img1: np.array, img2: np.array, min_matches: int = 10, threshold: float = 10000):
    # Create SIFT detector
    sift = cv2.SIFT_create()

    # Detect keypoints and compute descriptors
    keypoints1, descriptors1 = sift.detectAndCompute(img1, None)
    keypoints2, descriptors2 = sift.detectAndCompute(img2, None)

    # Create a matcher
    bf = cv2.FlannBasedMatcher()

    # Match descriptors
    matches = bf.knnMatch(descriptors1, descriptors2, k=2)

    # Apply ratio test to filter good matches
    good_matches = []
    for m, n in matches:
        if m.distance < 0.75 * n.distance:
            good_matches.append(m)

    # Draw matches on images
    img_matches = cv2.drawMatches(img1, keypoints1, img2, keypoints2, good_matches, None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)

    # Calculate a metric
    if len(good_matches) < min_matches:
        metric = np.inf
    else:
        metric_vector = np.array(sorted([_.distance for _ in good_matches])[:min_matches])
        metric = np.sum(metric_vector)
    print(f'Metric: {metric}')

    if metric < threshold:
        result = 'Same content!'
    else:
        result = 'Different content!'

    return metric, result, img_matches

# Tests

In [26]:
resolution = (640, 480)
# Load images
img1 = cv2.resize(cv2.imread('im1.jpg', cv2.IMREAD_GRAYSCALE), resolution)
img2 = cv2.resize(cv2.imread('im2.jpg', cv2.IMREAD_GRAYSCALE), resolution)
img3 = cv2.resize(cv2.imread('im3.jpg', cv2.IMREAD_GRAYSCALE), resolution)
img4 = cv2.resize(cv2.imread('IMG_20220214_133333313.jpg', cv2.IMREAD_GRAYSCALE), resolution)
img5 = cv2.resize(cv2.imread('IMG_20220214_144907525.jpg', cv2.IMREAD_GRAYSCALE), resolution)

for img_test_1, img_test_2 in itertools.combinations([img1, img2, img3, img4, img5], 2):
    metric, result, img_matches = image_comparator(img_test_1, img_test_2, threshold=1600)

    # print(result)
    # Display the result
    cv2.imshow(result + '. Press any key to continue', img_matches)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

Metric: 1510.5354232788086
Metric: 1893.7763977050781
Metric: 1649.1280670166016
Metric: 1454.1546020507812
Metric: inf
Metric: inf
Metric: inf
Metric: inf
Metric: inf
Metric: 712.1823463439941
